In [3]:
import pandas as pd 

## 1 
- Загрузите датасет VIX с использованием Pandas. 
- Убедитесь, что данные правильно интерпретируются, особенно дата.
- Преобразуйте данные, чтобы они имели нужный формат для анализа (например, установите колонку с датой как индекс).
- Постройте простой график временного ряда для колонки Close.
- Для выделения тренда добавьте скользящее среднее (например, на 7 дней) для значений Close и постройте график.
- Используйте метод resample для агрегации данных по месяцам и постройте график средних значений Close по месяцам.
- Используйте Seaborn для построения графика с доверительным интервалом (95% CI) для значений Close .
- Используйте метод IQR для определения выбросов и выделите их на графике. Для выбросов выберите значения, которые значительно отклоняются от основной тенденции.

In [6]:
df = pd.read_csv('../data/02-1-vix-daily.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df

,OPEN,HIGH,LOW,CLOSE
DATE,,,,
1990-01-02,17.24,17.24,17.24,17.24
1990-01-03,18.19,18.19,18.19,18.19
1990-01-04,19.22,19.22,19.22,19.22
1990-01-05,20.11,20.11,20.11,20.11
1990-01-08,20.26,20.26,20.26,20.26
...,...,...,...,...
2025-10-20,20.53,21.02,18.23,18.23
2025-10-21,18.65,18.89,17.40,17.87
2025-10-22,17.84,21.01,17.73,18.60
